In [1]:
import argparse
import os
from pprint import pprint


from anago.utils import load_data_and_labels,NERSequence
from anago.models import BiLSTMCRF,load_model,save_model
from anago.preprocessing import IndexTransformer#, ELMoTransformer
from anago.trainer import Trainer
from anago.tagger import Tagger
from lang_util.spell import SpellCorrector

spell = SpellCorrector()

Using TensorFlow backend.


In [6]:
##Input
dir_path = os.getcwd()
train_data = os.path.realpath(os.path.join(dir_path,"../data/train.iob"))
valid_data = os.path.realpath(os.path.join(dir_path,"../data/test.iob"))
weights_file = os.path.realpath(os.path.join(dir_path,"../models/weights.h5"))
params_file = os.path.realpath(os.path.join(dir_path,"../models/params.json"))
preprocessor_file = os.path.realpath(os.path.join(dir_path,"../models/preprocessor.bin"))
no_char_feature = 50

dropout = 0.5
no_use_crf = False
char_emb_size = 150
word_emb_size = 150
char_lstm_units = 150
word_lstm_units = 200

epochs = 5


In [5]:
print('Loading dataset...')
x_train, y_train = load_data_and_labels(train_data)
x_valid, y_valid = load_data_and_labels(valid_data)

print (x_train[0])
## set to one dimentional Y
Y = []
for yt in y_train:
    yx = []
    for y in yt:          
        yx.append(y.split(" ")[0])
    Y.append(yx)
y_train = Y
Y = []
for yt in y_valid:
    yx = []
    for y in yt:          
        yx.append(y.split(" ")[0])
    Y.append(yx)
y_valid = Y

Y = set()
for yt in y_train:
    for y in  (list(set(yt))):
        Y.add( y )


#save hasil spell
train_file = os.path.realpath(os.path.join(dir_path,"../data/train.txt"))
with open(train_file,'w') as tf:
    for xidx, xt in enumerate(x_train):
        prevlen  = len(xt)
        xspelled = spell.validate(" ".join(xt))
        xspelled = xspelled.split()
        if prevlen == len(xspelled):
            for idx,word in enumerate(xspelled):
                tf.write("%s\t%s\n" % (word, y_train[xidx][idx]) )
        else:
            print("%s translated to different length" % " ".join(xspelled))

        tf.write("\n")
    
valid_file = os.path.realpath(os.path.join(dir_path,"../data/valid.txt"))
with open(valid_file,'w') as vf:
    for xidx, xt in enumerate(x_valid):
        prevlen  = len(xt)
        xspelled = spell.validate(" ".join(xt))
        xspelled = xspelled.split()
        if prevlen == len(xspelled):
            for idx,word in enumerate(xspelled):
                tf.write("%s\t%s\n" % (word, y_valid[xidx][idx]) )
        else:
            print("%s translated to different length" % " ".join(xspelled))
        tf.write("\n")


Loading dataset...
['dok', 'ap', 'boleh', 'cabut', 'gigi', 'graham', 'atas']


In [15]:

print('Transforming datasets...')
p = IndexTransformer(use_char=no_char_feature)
p.fit(x_train, y_train)

print ("label size:",p.label_size )
print( len(list(Y))  )
print (p.char_vocab_size)
print (p.word_vocab_size)

Transforming datasets...
label size: 40
39
107
14501


In [13]:
print('Building a model.')
model = BiLSTMCRF(char_embedding_dim=char_emb_size,
                  word_embedding_dim=word_emb_size,
                  char_lstm_size=char_lstm_units,
                  word_lstm_size=word_lstm_units,
                  char_vocab_size=p.char_vocab_size,
                  word_vocab_size=p.word_vocab_size,
                  num_labels=p.label_size,
                  dropout=dropout,
                  use_char=no_char_feature,
                  use_crf=no_use_crf)
model, loss = model.build()
model.compile(loss=loss, optimizer='adam')


print('Training the model...')
trainer = Trainer(model, preprocessor=p)
trainer.train(x_train, y_train, x_valid, y_valid,epochs=epochs)




print('Saving the model...')
save_model(model,weights_file, params_file)    
p.save(preprocessor_file)

   

Building a model.
Training the model...
Epoch 1/5
203/203 [==============================] - 496s 2s/step - loss: 0.9511
 - f1: 47.74
             precision    recall  f1-score   support

       FOOD       0.00      0.00      0.00       125
      BBODY       0.69      0.61      0.65      1551
   ACTIVITY       0.24      0.13      0.17       533
   PREGNANT       0.84      0.78      0.81       293
       TEST       1.00      0.01      0.02       100
    SOMEONE       0.57      0.84      0.68       448
   DURATION       0.46      0.40      0.43       375
    DISEASE       0.62      0.03      0.06       894
    SYMPTOM       0.55      0.71      0.62      2817
ANTISYMPTOM       0.00      0.00      0.00        95
  TREATMENT       0.11      0.52      0.18       454
      ADULT       0.00      0.00      0.00        56
  LACTATION       0.00      0.00      0.00        77
       BABY       0.40      0.27      0.32       124
      CHILD       0.00      0.00      0.00        62
      WOMAN      

In [8]:

#model.score(x_valid,y_valid)

test =  "apakah demam menular ? dok saya umur 20 tahun. anak saya umur 8 bulan.  saya kemarin demam minum obat Cefadroxil 500 mg, Paracetamol 600 mg, Dicloflam 50 mg. saya sedang menyusui. saya sembuh, anak saya demam. apakah demam menular dok ? apakah demam karena obat yg saya minum dok ?"
#test = "setelah kuret dok saya mau nanya knp ya setelah kuret saya sering pusing sebelah"
test = "mata sebelah kiri bengkak dan di atas alis ada bisul Selamat malam dok , saya mau nanya , ibu saya awalnya di atas kening ada bintik merah , terus jadi jerawat lalu hilang , setelah itu mata kiri mama saya bengkak merah seperti digigit serangga , selang sehari di atas alis mata sebelah kiri tumbuh jerawat lalu jadi bisul dan sekarang mata ibu saya bengkak sampai dibawah mata , dan bengkak merah hampir gak bisa buka mata , kira kira itu kenapa dok ? Apakah karena bisul atau serangga atau yang lainnya ? Dan penanganan utama nya gimana dok ? Terimakasih"
test = ""
test = "kalau saat masa subur saya dan suami melakukan senggama terputus apakah mungkin masih bisa hamil"
#test = "raynaldy kelebihan berat badan yang bisa menyebabkan diabetes"
test = "masa subur sore dok. saya mau tanya, tanggal 05 juli 2018 nanti saya menikah. hari ini 21 juni 2018 saya haid hari pertama untuk periode juni. kira-kira masa subur saya tanggal berapa? karena saya dan calon suami berencana langsung memiliki momongan.."
test = "ambein dok saya berusia 20thn \
 namun ambeyen saya sudah sangat besar dan tidak dapat masuk kedalam dubur lagi \
 saya pernah periksakan namun hanya diberikan obat \
 tapi tidak ada reaksi \
 bagaimana ini dok \
 cara menyembuhkan wasir \
 tanpa ambil resiko operasi .\
 dan makanana apa saja yang harus dihindari"

test = "bekas jerawat dok ngilangin bekas jerawat pake apa yah? bekas jerawat saya di seblah pipi kanan gak ilang2.\
trus mau tanya jg kalau pelembab di pake setiap hari buat kulit yg ada bekas jerawat aman gk? wlpn itu gak keluar rumah"

test = "thyroid Dok, tiroid bisa menyebabkan jantung berdebar? Apa yg harus saya lakukan jika kambuh?"
test = "kelenjar tiroid dok kenapa yah semenjak saya memiliki kelenjar tiroid saya slalu merasa gelisah suasana hati kadang berubah apakah iitu ada hubunganya dengan benjolan saya"
tagger =  Tagger(model,p)
res = tagger.analyze(test)
pprint(res)

InvalidArgumentError: Reshape cannot infer the missing input size for an empty tensor unless all specified input sizes are non-zero
	 [[{{node time_distributed_1/Reshape}}]]